In [1]:
import pandas as pd
df = pd.read_csv("./markedpersonas/data/gpt4_main_generations.csv")
df.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,prompt_num,model,gender,race,prompt
0,0,0.0,1.0,0,A white man is an individual belonging to the ...,0,gpt4,M,a White,Describe a White man.
1,1,1.0,2.0,0,A white man refers to an individual who identi...,0,gpt4,M,a White,Describe a White man.
2,2,2.0,3.0,0,A white man typically refers to a male individ...,0,gpt4,M,a White,Describe a White man.
3,3,3.0,4.0,0,As I stand here looking at my reflection in th...,1,gpt4,M,a White,Describe a White man from the first-person per...
4,4,4.0,5.0,0,"As I glance in the mirror, I see a White man s...",1,gpt4,M,a White,Describe a White man from the first-person per...


In [2]:
import torch
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tqdm import tqdm

# Import transformers components
print("Loading transformers library...")
from transformers import BertTokenizer, BertModel

# Load BERT model and tokenizer
print("Loading BERT model...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

print("Model loaded successfully!")

Loading transformers library...
Loading BERT model...
Loading BERT model...
Model loaded successfully!
Model loaded successfully!


In [ ]:
# Function to get BERT embeddings
def get_bert_embedding(text):
    """Extract BERT embedding for a given text"""
    try:
        # Tokenize and prepare input
        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512, padding=True)
        
        # Get embeddings
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Use [CLS] token embedding as sentence representation
        embedding = outputs.last_hidden_state[:, 0, :].numpy()
        return embedding.flatten()
    except Exception as e:
        print(f"Error processing text: {e}")
        return None

In [3]:
# Extract embeddings for all generated texts
print("Extracting BERT embeddings...")
embeddings = []
texts = df['text'].tolist()
valid_indices = []

for i, text in tqdm(enumerate(texts), total=len(texts)):
    embedding = get_bert_embedding(str(text))
    if embedding is not None:
        embeddings.append(embedding)
        valid_indices.append(i)

embeddings = np.array(embeddings)
print(f"Successfully extracted {len(embeddings)} embeddings out of {len(texts)} texts")
print(f"Embeddings shape: {embeddings.shape}")

# Filter dataframe to only include valid embeddings
df = df.iloc[valid_indices].reset_index(drop=True)

Extracting BERT embeddings...


100%|██████████| 1350/1350 [01:43<00:00, 12.98it/s]

Successfully extracted 1350 embeddings out of 1350 texts
Embeddings shape: (1350, 768)


In [4]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

# Apply dimensionality reduction techniques
print("Applying dimensionality reduction...")

# 1. t-SNE
print("Running t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000)
embeddings_tsne = tsne.fit_transform(embeddings)

# 2. PCA
print("Running PCA...")
pca = PCA(n_components=2, random_state=42)
embeddings_pca = pca.fit_transform(embeddings)

# 3. UMAP
print("Running UMAP...")
reducer = umap.UMAP(n_components=2, random_state=42)
embeddings_umap = reducer.fit_transform(embeddings)

print("Dimensionality reduction complete!")

Applying dimensionality reduction...
Running t-SNE...
Running PCA...
Running UMAP...
Running PCA...
Running UMAP...


/mnt/f/CS7313-Group-Project/.venv/lib/python3.14/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Dimensionality reduction complete!


In [5]:
# Prepare data for visualization
# Add dimension reduction results to dataframe
df_viz = df.copy()
df_viz['tsne_x'] = embeddings_tsne[:, 0]
df_viz['tsne_y'] = embeddings_tsne[:, 1]
df_viz['pca_x'] = embeddings_pca[:, 0]
df_viz['pca_y'] = embeddings_pca[:, 1]
df_viz['umap_x'] = embeddings_umap[:, 0]
df_viz['umap_y'] = embeddings_umap[:, 1]

# Create hover text with truncated generation text
df_viz['hover_text'] = df_viz['text'].apply(lambda x: str(x)[:200] + '...' if len(str(x)) > 200 else str(x))

print("Data prepared for visualization!")

Data prepared for visualization!


In [6]:
# Visualization 1: t-SNE
fig_tsne = px.scatter(
    df_viz, 
    x='tsne_x', 
    y='tsne_y',
    color='model_name' if 'model_name' in df_viz.columns else None,
    hover_data=['hover_text'],
    title='BERT Embeddings Visualization - t-SNE',
    labels={'tsne_x': 't-SNE Component 1', 'tsne_y': 't-SNE Component 2'},
    width=900,
    height=700
)
fig_tsne.update_traces(marker=dict(size=8, opacity=0.7))
fig_tsne.show()

In [7]:
# Visualization 2: PCA
fig_pca = px.scatter(
    df_viz, 
    x='pca_x', 
    y='pca_y',
    color='model_name' if 'model_name' in df_viz.columns else None,
    hover_data=['hover_text'],
    title='BERT Embeddings Visualization - PCA',
    labels={'pca_x': f'PC1 ({pca.explained_variance_ratio_[0]:.2%})', 
            'pca_y': f'PC2 ({pca.explained_variance_ratio_[1]:.2%})'},
    width=900,
    height=700
)
fig_pca.update_traces(marker=dict(size=8, opacity=0.7))
fig_pca.show()

In [8]:
# Visualization 3: UMAP
fig_umap = px.scatter(
    df_viz, 
    x='umap_x', 
    y='umap_y',
    color='model_name' if 'model_name' in df_viz.columns else None,
    hover_data=['hover_text'],
    title='BERT Embeddings Visualization - UMAP',
    labels={'umap_x': 'UMAP Component 1', 'umap_y': 'UMAP Component 2'},
    width=900,
    height=700
)
fig_umap.update_traces(marker=dict(size=8, opacity=0.7))
fig_umap.show()

In [9]:
# Visualization 4: Side-by-side comparison
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('t-SNE', 'PCA', 'UMAP'),
    horizontal_spacing=0.1
)

# t-SNE
fig.add_trace(
    go.Scatter(
        x=df_viz['tsne_x'], 
        y=df_viz['tsne_y'],
        mode='markers',
        marker=dict(size=6, opacity=0.6, color='blue'),
        name='t-SNE',
        text=df_viz['hover_text'],
        hovertemplate='<b>Text:</b> %{text}<extra></extra>'
    ),
    row=1, col=1
)

# PCA
fig.add_trace(
    go.Scatter(
        x=df_viz['pca_x'], 
        y=df_viz['pca_y'],
        mode='markers',
        marker=dict(size=6, opacity=0.6, color='green'),
        name='PCA',
        text=df_viz['hover_text'],
        hovertemplate='<b>Text:</b> %{text}<extra></extra>'
    ),
    row=1, col=2
)

# UMAP
fig.add_trace(
    go.Scatter(
        x=df_viz['umap_x'], 
        y=df_viz['umap_y'],
        mode='markers',
        marker=dict(size=6, opacity=0.6, color='red'),
        name='UMAP',
        text=df_viz['hover_text'],
        hovertemplate='<b>Text:</b> %{text}<extra></extra>'
    ),
    row=1, col=3
)

fig.update_layout(
    title_text='BERT Embeddings: Comparison of Dimensionality Reduction Methods',
    showlegend=True,
    width=1800,
    height=600
)

fig.update_xaxes(title_text='Component 1', row=1, col=1)
fig.update_yaxes(title_text='Component 2', row=1, col=1)
fig.update_xaxes(title_text='Component 1', row=1, col=2)
fig.update_yaxes(title_text='Component 2', row=1, col=2)
fig.update_xaxes(title_text='Component 1', row=1, col=3)
fig.update_yaxes(title_text='Component 2', row=1, col=3)

fig.show()

In [10]:
# Additional analysis: Explained variance for PCA
print("PCA Explained Variance Ratio:")
print(f"PC1: {pca.explained_variance_ratio_[0]:.4f} ({pca.explained_variance_ratio_[0]*100:.2f}%)")
print(f"PC2: {pca.explained_variance_ratio_[1]:.4f} ({pca.explained_variance_ratio_[1]*100:.2f}%)")
print(f"Total: {sum(pca.explained_variance_ratio_):.4f} ({sum(pca.explained_variance_ratio_)*100:.2f}%)")

# Summary statistics
print("\n" + "="*50)
print("Summary:")
print("="*50)
print(f"Number of samples: {len(embeddings)}")
print(f"Embedding dimension: {embeddings.shape[1]}")
print(f"Reduced to: 2D for visualization")

PCA Explained Variance Ratio:
PC1: 0.2067 (20.67%)
PC2: 0.1430 (14.30%)
Total: 0.3497 (34.97%)

Summary:
Number of samples: 1350
Embedding dimension: 768
Reduced to: 2D for visualization
